In [1]:
import os 
# 这三行必须放在最上面，加载任何 huggingface 东西之前！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"  # 关键！国内加速镜站
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "300"  # 超时改成 5 分钟
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.expanduser("~/huggingface_cache")
from langchain_openai import ChatOpenAI
from langchain_classic.retrievers import EnsembleRetriever, BM25Retriever
from langchain_chroma import Chroma
from langchain_core.documents import Document
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

True

## Vector Retriever/Semantic Search

In [2]:
presist_directory = "db/chroma_db"
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

db = Chroma(
    persist_directory=presist_directory,
    embedding_function=embedding_model,
    collection_metadata={"hnsw:space": "cosine"},
)

query = "富贵的女儿叫什么名字？"

retrieval = db.as_retriever(search_kwargs={"k": 5})
relevant_docs = retrieval.invoke(query)

for i, doc in enumerate(relevant_docs):
    print("="*10,f"Document-{i+1}", "="*10)
    print(doc)

========== Document-1 ==========
page_content='"福贵。"
　　然后笑了笑，闭上了眼睛。过了一会，家珍又睁开眼睛问我："凤霞睡得好吗？'我起身看看凤霞，对她说：
　　"凤霞睡着了。"
　　那晚上家珍断断续续地说了好些话，到后来累了才睡着。
　　我却怎么都睡不着，心里七上八下的，家珍那样子像是好多了，可我老怕着是不是人常说的回光返照。我的手在她身上摸来摸去，还热着我才稍稍放心下来。
　　第二天我起床时，家珍还睡着，我想她昨晚上睡得晚，就没叫醒她，和凤霞喝了点粥下地去干活。那天收工早，我和凤霞回到家里时，我吓了一跳，家珍竟然坐在床上了，她是自己坐起来的。家珍看到我们进去，轻声说：
　　"福贵，我饿了，给我熬点粥。"
　　当时我傻站了很久，我怎么也想不到家珍会好起来了，家珍又叫了我一声，我才回过神来，我眼泪哗哗地流了出来，我忘了凤霞听不到，对凤霞说：
　　"全靠你，全靠你心里想着你娘不死。"
　　人只要想吃东西，那就没事了。过了一阵子，家珍坐在床上能干些针线活了，照这样下去，家珍没准又能下床走路。
　　我提着的心总算可以放下了，心里一踏实，人就病倒了。其实那病早就找到我了，有庆一死，家珍跟着是一副快去的样子，我顾不上病，也就不觉得。家珍没让医生说中，身体慢慢地好起来，我脑袋是越来越晕，直到有一天插秧时昏到了地上，被人抬回家，我才知道自己是病了。
　　我一病倒，凤霞可就苦了，床上躺着两个人，她又服侍我们又要下地挣工分。过了几天，我看着凤霞实在是太累，就跟家珍说好多了，拖着个病身体下田去干活，村里人见了我都吃了一惊，说：
　　"福贵，你头发全白了。"
　　我笑笑说："以前就白了。"
　　他们说："以前还有一半是黑的呢，就这么几天你的头发全白了。"
　　就那么几天，我老了许多，我以前的力气再也没有回来，干活时腰也酸了背也疼了，干得猛一些身上到处淌虚汗。' metadata={'source': 'docs/余华 活着.txt'}
========== Document-2 ==========
page_content='"爹，我死了埋在这里。"
　　我叹了口气对二喜说：
　　"这块就留给我吧，我怎么也会死在你前面的。"
　　埋掉了凤霞，孩子也可以从医院里抱出来了。二喜抱着他儿子走了十多里路来我家，把孩子放在床上，那孩子睁开眼睛时皱

## BM25 Retriever/Keyword Search

In [9]:
documents = db.get()["documents"]
len(documents), type(documents), documents[:2]

(137,
 list,
 ['余华《活着》\n\u3000\u3000\u3000\u3000\n前言\n\u3000\u3000一位真正的作家永远只为内心写作，只有内心才会真实地告诉他，他的自私、他的高尚是多么突出。内心让他真实地了解自己，一旦了解了自己也就了解了世界。很多年前我就明白了这个原则，可是要捍卫这个原则必须付出艰辛的劳动和长时期的痛苦，因为内心并非时时刻刻都是敞开的，它更多的时候倒是封闭起来，于是只有写作，不停地写作才能使内心敞开，才能使自己置身于发现之中，就像日出的光芒照亮了黑暗，灵感这时候才会突然来到。\n\u3000\u3000长期以来，我的作品都是源出于和现实的那一层紧张关系。我沉湎于想象之中，又被现实紧紧控制，我明确感受着自我的分裂，我无法使自己变得纯粹，我曾经希望自己成为一位童话作家，要不就是一位实实在在作品的拥有者，如果我能够成为这两者中的任何一个，我想我内心的痛苦将会轻微得多，可是与此同时我的力量也会削弱很多。\n\u3000\u3000事实上我只能成为现在这样的作家，我始终为内心的需要而写作，理智代替不了我的写作，正因为此，我在很长一段时间是一个愤怒和冷漠的作家。\n\u3000\u3000这不只是我个人面临的困难，几乎所有优秀的作家都处于和现实的紧张关系中，在他们笔下，只有当现实处于遥远状态时，他们作品中的现实才会闪闪发亮。应该看到，这过去的现实虽然充满魅力，可它已经蒙上了一层虚幻的色彩，那里面塞满了个人想象和个人理解。真正的现实，也就是作家生活中的现实，是令人费解和难以相处的。',
  '作家要表达与之朝夕相处的现实，他常常会感到难以承受，蜂拥而来的真实几乎都在诉说着丑恶和阴险，怪就怪在这里，为什么丑恶的事物总是在身边，而美好的事物却远在海角。换句话说，人的友爱和同情往往只是作为情绪来到，而相反的事实则是伸手便可触及。正像一位诗人所表达的：人类无法忍受太多的真实。也有这样的作家，一生都在解决自我和现实的紧张关系，福克纳是最为成功的例子，他找到了一条温和的途径，他描写中间状态的事物，同时包容了美好与丑恶，他将美国南方的现实放到了历史和人文精神之中，这是真正意义上的文学现实，因为它连接着过去和将来。\n\u3000\u3000一些不成功的作家也在描写现实，可他们笔下的现实说穿了只是一个环境，是固定的，死去的现实，他们看不到人是怎样走

In [10]:
bm25_retriever = BM25Retriever.from_texts(documents)
bm25_retriever.k = 5
bm25_docs = bm25_retriever.invoke(query)
bm25_docs

[Document(metadata={}, page_content='---全书完'),
 Document(metadata={}, page_content='老人和牛渐渐远去，我听到老人粗哑的令人感动的嗓音在远处传来，他的歌声在空旷的傍晚像风一样飘扬，老人唱道：\n\u3000\u3000少年去游荡，中年想掘藏，老年做和尚。\n\u3000\u3000炊烟在农舍的屋顶袅袅升起，在霞光四射的空中分散后消隐了。\n\u3000\u3000女人吆喝孩子的声音此起彼伏，一个男人挑着粪桶从我跟前走过，扁担吱呀吱呀一路响了过去。慢慢地，田野趋向了宁静，四周出现了模糊，霞光逐渐退去。\n\u3000\u3000我知道黄昏正在转瞬即逝，黑夜从天而降了。我看到广阔的土地袒露着结实的胸膛，那是召唤的姿态，就像女人召唤着她们的儿女，土地召唤着黑夜来临。'),
 Document(metadata={}, page_content='我便拉着牛走去，他们在后面乱哄哄地笑，我听到那个男人说：\n\u3000\u3000"今天合算，今天合算。"\n\u3000\u3000牛是通人性的，我拉着它往回走时，它知道是我救了它的命，身体老往我身上靠，亲热得很，我对它说：\n\u3000\u3000"你呀，先别这么高兴，我拉你回去是要你干活，不是把你当爹来养着的。"\n\u3000\u3000我拉着牛回到村里，村里人全围上来看热闹，他们都说我老糊涂了，买了这么一头老牛回来，有个人说：\n\u3000\u3000"福贵，我看它年纪比你爹还大。"\n\u3000\u3000会看牛的告诉我，说它最多只能活两年三年的，我想两三年足够了，我自己恐怕还活不到这么久。谁知道我们都活到了今天，村里人又惊又奇，就是前两天，还有人说我们是--"两个老不死。"\n\u3000\u3000牛到了家，也是我家里的成员了，该给它取个名字，想来想去还是觉得叫它福贵好。定下来叫它福贵，我左看右看都觉得它像我，心里美滋滋的，后来村里人也开*妓滴颐橇礁龊*像，我嘿嘿笑，心想我早就知道它像我了。\n\u3000\u3000福贵是好样的，有时候嘛，也要偷偷懒，可人也常常偷懒，就不要说是牛了。我知道什么时候该让它干活，什么时候该让它歇一歇，只要我累了，我知道它也累了，就让它歇一会，我歇得来精神了，那它也该干活了。\n\u3000\u3

## Hybrid Retriever

In [11]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[retrieval, bm25_retriever],
    weights=[0.5, 0.5]
)
hybrid_result = hybrid_retriever.invoke(query)
for i, doc in enumerate(hybrid_result):
    print("="*20, f"Document-{i+1}", "="*20)
    print(doc)

==================== Document-1 ====================
page_content='"福贵。"
　　然后笑了笑，闭上了眼睛。过了一会，家珍又睁开眼睛问我："凤霞睡得好吗？'我起身看看凤霞，对她说：
　　"凤霞睡着了。"
　　那晚上家珍断断续续地说了好些话，到后来累了才睡着。
　　我却怎么都睡不着，心里七上八下的，家珍那样子像是好多了，可我老怕着是不是人常说的回光返照。我的手在她身上摸来摸去，还热着我才稍稍放心下来。
　　第二天我起床时，家珍还睡着，我想她昨晚上睡得晚，就没叫醒她，和凤霞喝了点粥下地去干活。那天收工早，我和凤霞回到家里时，我吓了一跳，家珍竟然坐在床上了，她是自己坐起来的。家珍看到我们进去，轻声说：
　　"福贵，我饿了，给我熬点粥。"
　　当时我傻站了很久，我怎么也想不到家珍会好起来了，家珍又叫了我一声，我才回过神来，我眼泪哗哗地流了出来，我忘了凤霞听不到，对凤霞说：
　　"全靠你，全靠你心里想着你娘不死。"
　　人只要想吃东西，那就没事了。过了一阵子，家珍坐在床上能干些针线活了，照这样下去，家珍没准又能下床走路。
　　我提着的心总算可以放下了，心里一踏实，人就病倒了。其实那病早就找到我了，有庆一死，家珍跟着是一副快去的样子，我顾不上病，也就不觉得。家珍没让医生说中，身体慢慢地好起来，我脑袋是越来越晕，直到有一天插秧时昏到了地上，被人抬回家，我才知道自己是病了。
　　我一病倒，凤霞可就苦了，床上躺着两个人，她又服侍我们又要下地挣工分。过了几天，我看着凤霞实在是太累，就跟家珍说好多了，拖着个病身体下田去干活，村里人见了我都吃了一惊，说：
　　"福贵，你头发全白了。"
　　我笑笑说："以前就白了。"
　　他们说："以前还有一半是黑的呢，就这么几天你的头发全白了。"
　　就那么几天，我老了许多，我以前的力气再也没有回来，干活时腰也酸了背也疼了，干得猛一些身上到处淌虚汗。' metadata={'source': 'docs/余华 活着.txt'}
==================== Document-2 ====================
page_content='---全书完'
==================== Document-3 ====================
page_c

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage
model = ChatOpenAI(model="doubao-seed-1-6-lite-251015")
combined_input = f'''请结合文档中的内容，回答我的问题。
文档内容如下：
{chr(10).join([f"{doc.page_content}" for doc in hybrid_result])}
我的问题是：{query}
'''

messages = [
    SystemMessage(content="你是一个乐于助人的助手！"),
    HumanMessage(content=combined_input),
]
results = model.invoke(messages)
print(results.content)

富贵的女儿叫凤霞。文档中多次提及"凤霞"，如"凤霞睡得好吗？""凤霞死后不到三个月"等，明确其为富贵的女儿。
